# **ZMSI_Lab_5**
Wykorzystanie pretrenowanych modeli uczenia głębokiego do trenowania nowych modeli

$\color{red}{Uwaga:}$

1- proszę pamiętać o zmianie słowa ***Album*** w nazwie pliku i uzupełnieniu swoich danych powyżej tego pola;

2- kluczowe polecenia proszę opatrzyć komentarzem;

3- wnioski, pod zadaniem, wpisujemy w polu tekstowym.

In [ ]:
# Podpisywanie pracy
NN = input("Podaj Imię i Nazwisko: ")
ALBUM = input("Podaj numer albumu: ")

Podaj Imię i Nazwisko: Kacper Gąsowski
Podaj numer albumu: 154845


In [ ]:
# Ocena pracy. Proszę nie zmieniać komórek z kodem pomocniczym do oceny.
punkty = []
max = int(45)

### **Zad 5.** Zbuduj, i przetestuj modele (zbudowane z zamrożonych warstw konwolucyjnych pretrenowanego modelu VGG16 na zbiorze ImageNet oraz z warstw gęstych) klasyfikujące przykładowe obrazy ze zbioru Stanford Dogs Dataset, wykorzystujące generatory danych obrazowych z poprzednich zajęć.

### **Zad 5.1.** Zbuduj generatory.

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


data_dir = '/content/drive/My Drive/KUL/Obrazy/Images'
all_folders = os.listdir(data_dir)
selected_folders = random.sample(all_folders, 2)  # Wybieramy dwa losowe foldery

# Nowa ścieżka do danych z wybranymi folderami
selected_data_dir = '/content/selected_data'
os.makedirs(selected_data_dir, exist_ok=True)

for folder in selected_folders:
    src = os.path.join(data_dir, folder)
    dst = os.path.join(selected_data_dir, folder)
    os.symlink(src, dst)  # Tworzymy dowiązania symboliczne do wybranych folderów

img_height, img_width = 224, 224
batch_size = 32
num_classes = 2  # Liczba klas to teraz 2

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% danych jako walidacyjnych
)

train_generator = train_datagen.flow_from_directory(
    selected_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    selected_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Accuracy on validation set: {val_accuracy:.2f}')


Found 319 images belonging to 2 classes.
Found 78 images belonging to 2 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 125s 5s/step - accuracy: 0.5508 - loss: 2.8123 - val_accuracy: 0.8125 - val_loss: 0.3593
Epoch 2/10
1/9 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.6250 - loss: 0.6486

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


9/9 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.6250 - loss: 0.6486 - val_accuracy: 0.6429 - val_loss: 1.1417
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 387ms/step - accuracy: 0.7525 - loss: 0.8762 - val_accuracy: 0.8125 - val_loss: 0.4237
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.8438 - loss: 0.4137 - val_accuracy: 0.7143 - val_loss: 0.7983
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 346ms/step - accuracy: 0.8228 - loss: 0.4809 - val_accuracy: 0.8281 - val_loss: 0.3234
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.8438 - loss: 0.3997 - val_accuracy: 0.7857 - val_loss: 0.4283
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 338ms/step - accuracy: 0.8918 - loss: 0.2643 - val_accuracy: 0.8125 - val_loss: 0.3769
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8710 - loss: 0.2796 - val_accuracy: 0.9286 - val_loss: 0.0938
Epoch 9/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 408ms/step - accuracy: 0.8900 - loss: 0.2245 - val_accuracy: 0.8594 - val_loss: 0.3216
Epoch

In [ ]:
# Maksymalna wartość punktów za Zad 5.1: 15
punkty.append(15)

### **Zad. 5.2.** Wykorzystaj jeden prosty generator bez augmentacji. Ze zbioru pobranego z sieci wybierz 2-3 rasy spośród 120. Przełącz się w środowisku chmury na procesor graficzny.

In [ ]:
pip install --upgrade tensorflow


In [ ]:
# Sprawdzamy liczbę klas w folderze
for folder in selected_folders:
    folder_path = os.path.join(selected_data_dir, folder)
    num_images = len(os.listdir(folder_path))
    print(f'Folder: {folder}, Liczba obrazów: {num_images}')


Folder: n02086646-Blenheim_spaniel, Liczba obrazów: 188
Folder: n02097474-Tibetan_terrier, Liczba obrazów: 206
Folder: n02106166-Border_collie, Liczba obrazów: 150


In [3]:
import os
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Ścieżka do folderu z obrazami
data_dir = '/content/drive/My Drive/KUL/Obrazy/Images'

# Wybieramy losowo 3 foldery (klasy)
all_folders = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]
selected_folders = random.sample(all_folders, 3)

# Tworzymy nowy katalog z dowiązaniami do wybranych folderów
selected_data_dir = '/content/selected_data_no_aug'
os.makedirs(selected_data_dir, exist_ok=True)

# Tworzenie dowiązań symbolicznych do obrazów w wybranych folderach
for folder in selected_folders:
    src_folder = os.path.join(data_dir, folder)
    dst_folder = os.path.join(selected_data_dir, folder)
    os.makedirs(dst_folder, exist_ok=True)

    for filename in os.listdir(src_folder):
        src = os.path.join(src_folder, filename)
        dst = os.path.join(dst_folder, filename)
        os.symlink(src, dst)  # Tworzymy dowiązania symboliczne do obrazów w wybranych folderach

# Sprawdzamy liczbę klas na podstawie liczby wybranych folderów
num_classes = len(selected_folders)

# Parametry treningu
img_height, img_width = 224, 224
batch_size = 32

# Tworzymy generator z danymi, bez augmentacji, tylko reskalowanie
simple_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Generatory treningowe i walidacyjne
train_generator = simple_datagen.flow_from_directory(
    selected_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = simple_datagen.flow_from_directory(
    selected_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Ładowanie modelu VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Zatrzymujemy trenowanie warstw bazowego modelu

# Tworzenie własnego modelu na bazie VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)  # Liczba klas zależna od folderów

# Kompilacja modelu
model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Liczba epok
epochs = 10

# Trening modelu
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)


Found 366 images belonging to 3 classes.
Found 91 images belonging to 3 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11/11 ━━━━━━━━━━━━━━━━━━━━ 105s 4s/step - accuracy: 0.3672 - loss: 3.9326 - val_accuracy: 0.6406 - val_loss: 1.0746
Epoch 2/10
 1/11 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 0.5000 - loss: 2.7144

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


11/11 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.5000 - loss: 2.7144 - val_accuracy: 0.7037 - val_loss: 0.7443
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 23s 181ms/step - accuracy: 0.6332 - loss: 1.5937 - val_accuracy: 0.6250 - val_loss: 1.3502
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7857 - loss: 0.5752 - val_accuracy: 0.7037 - val_loss: 0.9634
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 214ms/step - accuracy: 0.7840 - loss: 0.6020 - val_accuracy: 0.7344 - val_loss: 0.6326
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9375 - loss: 0.3813 - val_accuracy: 0.6667 - val_loss: 0.6445
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 194ms/step - accuracy: 0.8895 - loss: 0.3194 - val_accuracy: 0.8281 - val_loss: 0.5128
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.9286 - loss: 0.3934 - val_accuracy: 0.6296 - val_loss: 0.8546
Epoch 9/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 179ms/step - accuracy: 0.9024 - loss: 0.2688 - val_accuracy: 0.7500 - val_loss

In [ ]:
# Maksymalna wartość punktów za Zad 5.2: 15
punkty.append(15)

### **Zad. 5.2.** Wykorzystaj możliwość wykorzystania wielu generatorów w jednym procesie uczenia. W ostatniej komórce tekstowej przedstaw wnioski z ćwiczeń.

In [ ]:
import os
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

data_dir = '/content/drive/My Drive/KUL/Obrazy/Images'
all_folders = os.listdir(data_dir)

# Wybieramy 2-3 losowe foldery dla generatora treningowego i walidacyjnego
train_folders = random.sample(all_folders, 3)
validation_folders = random.sample(all_folders, 3)

# Tworzymy nowe katalogi na wybrane foldery
train_data_dir = '/content/train_data_no_aug'
validation_data_dir = '/content/validation_data_no_aug'

os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(validation_data_dir, exist_ok=True)

for folder in train_folders:
    src = os.path.join(data_dir, folder)
    dst = os.path.join(train_data_dir, folder)
    os.symlink(src, dst)

for folder in validation_folders:
    src = os.path.join(data_dir, folder)
    dst = os.path.join(validation_data_dir, folder)
    os.symlink(src, dst)

img_height, img_width = 224, 224
batch_size = 32
num_classes = len(train_folders)  # Liczba klas na podstawie liczby wybranych folderów

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)



Found 465 images belonging to 3 classes.
Found 596 images belonging to 3 classes.
Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 698s 44s/step - accuracy: 0.3451 - loss: 6.4456 - val_accuracy: 0.4045 - val_loss: 1.6420
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.4062 - loss: 2.1261 - val_accuracy: 0.3500 - val_loss: 2.1354
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 367ms/step - accuracy: 0.5619 - loss: 1.3963 - val_accuracy: 0.4479 - val_loss: 1.1574
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8438 - loss: 0.4342 - val_accuracy: 0.3500 - val_loss: 1.2148
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 411ms/step - accuracy: 0.7554 - loss: 0.5796 - val_accuracy: 0.4861 - val_loss: 1.1142
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7812 - loss: 0.5925 - val_accuracy: 0.3500 - val_loss: 1.2602
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 374ms/step - accuracy: 0.8660 - loss: 0.3837 - val_accuracy: 0.4878 - val_loss: 1.2281
Epoch 8/10
14/14 ━━━━━━━━━━━

**Dokładność trenowania i walidacji**: Przy ograniczonej liczbie epok (10) i małym podzbiorze danych można oczekiwać, że dokładność osiągnie przyzwoity poziom, choć może nie być optymalna. Transfer learning pozwala na szybkie uzyskanie akceptowalnych wyników nawet przy małej liczbie obrazów.

**Problemy z generalizacją**: Ponieważ dane treningowe obejmują tylko część klas, model może nie uogólniać się dobrze na inne kategorie obrazów. Dobór losowych klas może prowadzić do tego, że model działa dobrze na testowych próbkach z tych samych kategorii, ale jego skuteczność może spaść, gdy zobaczy obrazy spoza tego podzbioru klas.

**Nadmierne dopasowanie**: Jeśli dokładność na zbiorze treningowym znacznie przewyższa dokładność walidacyjną, może to wskazywać na nadmierne dopasowanie. Zastosowanie Dropout oraz zwiększenie liczby danych treningowych lub ich augmentacja mogłoby zredukować ten problem.



In [ ]:
# Maksymalna wartość punktów za Zad 5.3: 15
punkty.append(15)

In [ ]:
razem = sum(punkty)
razem_pr = razem/max
if razem_pr >= 0.90:
  ocena = 5
elif razem_pr >= 0.80:
  ocena = 4.5
elif razem_pr >= 0.70:
  ocena = 4
elif razem_pr >= 0.60:
  ocena = 3.5
elif razem_pr >= 0.50:
  ocena = 3
else:
  ocena = 2

print("Punktacja za kolejne zadania: ", punkty)
print("Suma punktów: ", razem)
print("Udział procentowy: ", razem_pr*100, "%")
print("Ocena: ", ocena)

Punktacja za kolejne zadania:  [15, 15, 15]
Suma punktów:  45
Udział procentowy:  100.0 %
Ocena:  5
